# Format transformation and Templates
| - | VOC | YOLO | COCO |
| - | - | - | - |
| file | .xml per img | .txt per img | .json for all img (D['images'].id==D['annotations'].image_id) |
| format | xmin,ymin,xmax,ymax | cx,cy,w,h | xmin,ymin,w,h |
| type | int | float | int |
| classes | - | classes.txt | D['categories'] |
| raw shape | height,width | - | D['images'] |

In [1]:
import os, glob, re, json
import cv2 # yolo2others only

if True:
    xml0="\
<annotation>\n\
	<folder>folderX</folder>\n\
	<filename>filenameX</filename>\n\
	<path>pathX</path>\n\
	<source>\n\
		<database>Unknown</database>\n\
	</source>\n\
	<size>\n\
		<width>widthX</width>\n\
		<height>heightX</height>\n\
		<depth>3</depth>\n\
	</size>\n\
	<segmented>0</segmented>\n\
" # folderX,filenameX,pathX,widthX,heightX need to be replaced
    obj0="\
	<object>\n\
		<name>nameX</name>\n\
		<pose>Unspecified</pose>\n\
		<truncated>0</truncated>\n\
		<difficult>0</difficult>\n\
		<bndbox>\n\
			<xmin>xminX</xmin>\n\
			<ymin>yminX</ymin>\n\
			<xmax>xmaxX</xmax>\n\
			<ymax>ymaxX</ymax>\n\
		</bndbox>\n\
	</object>\n\
" # nameX,xmin,ymin,xmax,ymax need to be replaced
    end0="</annotation>"
    
def boxAny2Voc(srcType, b1, b2, b3, b4, width=None, height=None):
    if srcType=="voc": # b1,b2,b3,b4 = xmin,ymin,xmax,ymax
        xmin, ymin, xmax, ymax = int(b1), int(b2), int(b3), int(b4)
    elif srcType=="yoloFloat": # b1,b2,b3,b4 = cx,cy,w,h
        xmin = int((float(b1)-float(b3)/2)*float(width))
        ymin = int((float(b2)-float(b4)/2)*float(height))
        xmax = int((float(b1)+float(b3)/2)*float(width))
        ymax = int((float(b2)+float(b4)/2)*float(height))
    elif srcType=="yoloInt": # b1,b2,b3,b4 = cx,cy,w,h
        xmin = int(int(b1)-int(b3)/2)
        ymin = int(int(b2)-int(b4)/2)
        xmax = int(int(b1)+int(b3)/2)
        ymax = int(int(b2)+int(b4)/2)
    elif srcType=="coco": # b1,b2,b3,b4 = xmin,ymin,w,h
        xmin, ymin, xmax, ymax = int(b1), int(b2), int(b1)+int(b3), int(b2)+int(b4)
    else:
        raise KeyError(f"{srcType} Not found")
    return xmin, ymin, xmax, ymax

def boxVoc2Any(desType, xmin, ymin, xmax, ymax, width=None, height=None):
    if desType=="voc":
        return int(xmin), int(ymin), int(xmax), int(ymax)
    elif desType=="yoloFloat":
        cx = round((int(xmin)+int(xmax))/2/float(width),6)
        cy = round((int(ymin)+int(ymax))/2/float(height),6)
        w  = round((int(xmax)-int(xmin))/float(width),6)
        h  = round((int(ymax)-int(ymin))/float(height),6)
        return cx, cy, w, h
    elif desType=="yoloInt":
        cx = int((int(xmin)+int(xmax))/2)
        cy = int((int(ymin)+int(ymax))/2)
        w  = int((int(xmax)-int(xmin)))
        h  = int((int(ymax)-int(ymin)))
        return cx, cy, w, h
    elif desType=="coco":
        xmin = int(xmin)
        ymin = int(ymin)
        w    = int(xmax)-int(xmin)
        h    = int(ymax)-int(ymin)
        return xmin, ymin, w, h
    else:
        raise KeyError(f"{desType} Not found")

# Label data transformation tool

In [3]:
# yolo means yoloFloat
def voc2yolo(sourceFolder, destFolder, classList):
    with open(f"{destFolder}/classes.txt","w") as f:
        for c in classList:
            f.writelines(f"{c}\n")
    for xmlPath in glob.glob(f"{sourceFolder}/*.xml"):
        xml = open(xmlPath,"r").read()
        width  = int(re.findall("<width>([0-9]*)</width>",xml)[0])
        height = int(re.findall("<height>([0-9]*)</height>",xml)[0])
        nameL = re.findall("<name>(.*)</name>",xml)
        xminL = re.findall("<xmin>(.*)</xmin>",xml)
        yminL = re.findall("<ymin>(.*)</ymin>",xml)
        xmaxL = re.findall("<xmax>(.*)</xmax>",xml)
        ymaxL = re.findall("<ymax>(.*)</ymax>",xml)
        with open(f"{destFolder}/{xmlPath.split('/')[-1].replace('.xml','.txt')}", "w") as f:
            for name,xmin,ymin,xmax,ymax in zip(nameL,xminL,yminL,xmaxL,ymaxL):
                id = classList.index(name)
                cx, cy, w, h = boxVoc2Any("yoloFloat",xmin,ymin,xmax,ymax,width,height) #
                pad = lambda s:str(s)+'0'*(8-len(str(s)))
                f.writelines(f"{id} {pad(cx)} {pad(cy)} {pad(w)} {pad(h)}\n")

def voc2coco(sourceFolder, destFolder, classList):
    D = {"images":[], "annotations":[], "categories": []}
    D["categories"] = [ {"supercategory":"none","id":i,"name":className} for i,className in enumerate(classList,1) ]
    for id,xmlPath in enumerate(sorted(glob.glob(f"{sourceFolder}/*.xml"))):
        xml = open(xmlPath,"r").read()
        filename = xmlPath.split('/')[-1].replace('.xml','.jpg')
        height = int(re.findall("<height>([0-9]*)</height>",xml)[0])
        width  = int(re.findall("<width>([0-9]*)</width>",xml)[0])
        nameL  = re.findall("<name>(.*)</name>",xml)
        xminL  = re.findall("<xmin>(.*)</xmin>",xml)
        yminL  = re.findall("<ymin>(.*)</ymin>",xml)
        xmaxL  = re.findall("<xmax>(.*)</xmax>",xml)
        ymaxL  = re.findall("<ymax>(.*)</ymax>",xml)
        D["images"].append( {"file_name":filename,"height":height,"width":width,"id":id} )
        for i,(name,xmin,ymin,xmax,ymax) in enumerate(zip(nameL,xminL,yminL,xmaxL,ymaxL),1):
            xmin, ymin, w, h = boxVoc2Any("coco",xmin,ymin,xmax,ymax) #
            catid= classList.index(name)+1
            D["annotations"].append( {"area":w*h,"iscrowd":0,"bbox":[xmin,ymin,w,h],"category_id":catid,\
                "ignore":0,"segmentation":[],"image_id":id,"id":i} )
    with open(f"{destFolder}/labels.json", "w") as f:
        json.dump(D,f)
                
def yolo2voc(sourceFolder, destFolder):
    global xml0, obj0, end0
    with open(f"{sourceFolder}/classes.txt","r") as f:
        D = {str(i):key[:-1] for i,key in enumerate(f.readlines())} # e.g. D={0:'dog',1:'cat'}
    for txtPath in glob.glob(f"{sourceFolder}/*.txt"):
        folder   = destFolder.split('/')[-1] if "/" in destFolder else os.getcwd().split('/')[-1]
        filename = txtPath.split('/')[-1].replace('.txt','.jpg')
        path     = f"{os.path.abspath(destFolder)}/{filename}"
        img = cv2.imread(txtPath.replace('.txt','.jpg'))
        if type(img)!=type(None):
            height, width, _ = cv2.imread(txtPath.replace('.txt','.jpg')).shape
        else:
            continue
        xml = xml0.replace('folderX',folder).replace('filenameX',filename).replace('pathX',path).\
            replace('widthX',str(width)).replace('heightX',str(height))
        for yoloLine in open(txtPath).readlines():
            id, cx, cy, w, h = yoloLine.split(" ")
            xmin, ymin, xmax, ymax = boxAny2Voc("yoloFloat",cx,cy,w,h,width,height) #
            obj = obj0.replace('nameX',D[id]).replace('xminX',str(xmin)).replace('yminX',str(ymin)).\
                replace('xmaxX',str(xmax)).replace('ymaxX',str(ymax))
            xml+=obj
        xml+=end0
        with open(f"{destFolder}/{filename.replace('.jpg','.xml')}",'w') as f:
            f.write(xml)
        
def coco2voc(sourceFolder, destFolder):
    global xml0, obj0, end0
    D = json.load( open(f"{sourceFolder}/labels.json","r") )
    for imgD in D['images']:
        folder   = destFolder.split('/')[-1] if "/" in destFolder else os.getcwd().split('/')[-1]
        filename = imgD['file_name']
        path     = f"{os.path.abspath(destFolder)}/{filename}"
        height   = imgD['height']
        width    = imgD['width']
        xml = xml0.replace('folderX',folder).replace('filenameX',filename).replace('pathX',path).\
            replace('widthX',str(width)).replace('heightX',str(height))
        classDict = { catD['id']:catD['name'] for catD in D['categories'] } # classDict={0:'dog', 1:'cat'}
        for annotD in filter(lambda d:d['image_id']==imgD['id'], D['annotations']):
            cname = classDict[ annotD['category_id'] ]
            xmin, ymin, w, h = annotD['bbox']
            xmin, ymin, xmax, ymax = boxAny2Voc("coco",xmin,ymin,w,h) #
            obj = obj0.replace('nameX',cname).replace('xminX',str(xmin)).replace('yminX',str(ymin)).\
                replace('xmaxX',str(xmax)).replace('ymaxX',str(ymax))
            xml+=obj
        xml+=end0
        with open(f"{destFolder}/{filename.replace('.jpg','.xml')}",'w') as f:
            f.write(xml)

In [4]:
yolo2voc("./","./")

In [5]:
voc2yolo("./","./",["PlasticContainer","PaperContaine"])

In [6]:
voc2coco("./","./",["PlasticContainer","PaperContaine"])

In [7]:
coco2voc("./","./")

# Pseudo labeling tool

In [30]:
# support yoloFloat and voc only
def arr2Label(rawImgPath, bboxes, cids, classList, srcType="yoloFloat", desType="yoloFloat", destFolder="./pseudoLabel"):
    os.system(f"mkdir {destFolder}")
    height, width, _ = cv2.imread(rawImgPath).shape
    rawImgName = rawImgPath.split('/')[-1] if "/" in rawImgPath else os.getcwd().split('/')[-1]
    if desType=="voc":
        global xml0, obj0, end0
        folder   = destFolder.split('/')[-1] if "/" in destFolder else os.getcwd().split('/')[-1]
        xml = xml0.replace('folderX',folder).replace('filenameX',rawImgName).replace('pathX',rawImgPath).\
            replace('widthX',str(width)).replace('heightX',str(height))
        for (b1,b2,b3,b4),cid in zip(bboxes,cids):
            xmin, ymin, xmax, ymax = boxAny2Voc(srcType, b1, b2, b3, b4, width, height)
            cname = classList[cid]
            obj = obj0.replace('nameX',cname).replace('xminX',str(xmin)).replace('yminX',str(ymin)).\
                replace('xmaxX',str(xmax)).replace('ymaxX',str(ymax))
            xml+=obj
        xml+=end0
        with open(f"{destFolder}/{rawImgName.replace('.jpg','.xml')}",'w') as f:
            f.write(xml)  
    elif desType=="yoloFloat":
        with open(f"{destFolder}/classes.txt","w") as f:
            for c in classList:
                f.writelines(f"{c}\n")
        with open(f"{destFolder}/{rawImgName.replace('.jpg','.txt')}", "w") as f:
            for (b1,b2,b3,b4),cid in zip(bboxes,cids):
                xmin, ymin, xmax, ymax = boxAny2Voc(srcType, b1, b2, b3, b4, width, height)
                cx, cy, w, h = boxVoc2Any(desType, xmin, ymin, xmax, ymax, width, height)
                pad = lambda s:str(s)+'0'*(8-len(str(s)))
                f.writelines(f"{cid} {pad(cx)} {pad(cy)} {pad(w)} {pad(h)}\n")
    else:
        raise KeyError(f"Unknown desType:{desType}")

In [31]:
boxesYoloFloat = [[0.308203, 0.699306, 0.141406, 0.234722], [0.800000, 0.479167, 0.237500, 0.283333], [0.858594, 0.831944, 0.165625, 0.216667],
                  [0.062891, 0.294444, 0.125781, 0.266667], [0.232813, 0.504167, 0.120313, 0.163889], [0.123438, 0.131944, 0.221875, 0.163889],
                  [0.748437, 0.879167, 0.134375, 0.216667], [0.557422, 0.116667, 0.144531, 0.225000], [0.425391, 0.380556, 0.260156, 0.266667],
                  [0.404297, 0.860417, 0.164844, 0.215278], [0.204297, 0.677778, 0.078906, 0.363889], [0.134375, 0.572917, 0.096875, 0.165278]]
cids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
classList = ["PlasticContainer","PaperContaine"]

arr2Label("./C0025_00505.jpg", boxesYoloFloat, cids, classList, srcType="yoloFloat", desType="yoloFloat")
arr2Label("./C0025_00505.jpg", boxesYoloFloat, cids, classList, srcType="yoloFloat", desType="voc")